In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import json


from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from skimage import transform


from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile,assemble
from qiskit import Aer, execute
from qiskit.extensions import Initialize
from qiskit.tools.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.quantum_info import partial_trace, Statevector, random_statevector, Operator, SparsePauliOp
from qiskit_textbook.tools import simon_oracle

from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit.circuit.library import TwoLocal, NLocal, RealAmplitudes, EfficientSU2
from qiskit.circuit.library import HGate, RXGate, RYGate, RZGate, CXGate, CRXGate, CRZGate
from qiskit.circuit.library import RealAmplitudes

from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.algorithms.classifiers import VQC, NeuralNetworkClassifier
from qiskit_machine_learning.neural_networks import EstimatorQNN

from qiskit.algorithms.optimizers import COBYLA, GradientDescent, ADAM
from qiskit.primitives import Sampler
from qiskit.utils import algorithm_globals


from IPython.display import clear_output

algorithm_globals.random_seed = 42

In [ ]:
def create_bell_state(circuit, q1, q2):
    circuit.h(q1)
    circuit.cnot(q1, q2)

def create_bell_state(circuit, register):
    circuit.h(register[0])
    circuit.cnot(register[0], register[1])

def teleport_gates(circuit, alice_qbit, alice_ent):
    circuit.cnot(alice_qbit, alice_ent)
    circuit.h(alice_qbit)


def swap(circuit, q1, q2):
    circuit.cnot(q1, q2)
    circuit.cnot(q2,q1)
    circuit.cnot(q1, q2)

# Data Encoding

## Basis state

Feature binarie. Dati i campioni assegnamo equo amplitude e nulla alle altre. Dato che 

$
\sum_{i=1}^{2^N} \alpha_i^2 = 1
$

la somma delle amplitude per ogni stato deve essere uguale ad uno, e vogliamo assegnare equa amplitude (solo per i dati presenti), questa sarà (con N numero campioni):

$
1 = \sum_{i=1}^{N} \alpha^2 = N\alpha^2  \rightarrow \alpha = \frac{1}{\sqrt{N}}
$

In [ ]:
N_FEATURES = 3
DATA = ['101', '111', '000']

state_preparation = [0] * (2**N_FEATURES)
for e in DATA:
    state_preparation[int(e, 2)] = 1/math.sqrt(len(DATA))

print(state_preparation)

qc = QuantumCircuit(N_FEATURES, N_FEATURES)
qc.initialize(state_preparation, range(N_FEATURES))


qc.decompose().draw(output="mpl")

## Amplitude

Per rappresentare un intero dataset concateno i dati di ogni elemento. Per un dataset in $\mathbb{R}^{n\times n}$ avrò quindi bisogno di   


$
N_{qbit} = n^2 log_2(n^2)
$


Dato quindi un dataset $X = (\alpha_1, \dots , \alpha_n)$ necessario rinormalizzare i dati in modo tale che, data una costante di normalizzazione $k$:

$
\sum_{i=1}^{n} |(k\alpha_i)^2| = 1
$
si ha quindi

$
k = \sqrt{\frac{1} {\sum_{i=1}^{n} |\alpha_i^2|}}
$

In [ ]:
DATA = np.array([[5,2,3],[1,3,1]])
N_QBIT = math.ceil(math.log2(DATA.size))
print(N_QBIT)
value =  math.sqrt(1/(np.sum(DATA.flatten()**2)))

state_preparation = np.append(DATA.flatten() * value, [0] * (2**N_QBIT - DATA.size))

qc = QuantumCircuit(N_QBIT, N_QBIT)
qc.initialize(state_preparation, range(N_QBIT))

qc.decompose().draw(output="mpl")

# SVM




## Kernel Inner product

In [ ]:
digits = datasets.load_digits(n_class=2)

fig, axs = plt.subplots(1, 2, figsize=(6,3))
axs[0].set_axis_off()
axs[0].imshow(digits.images[0], cmap=plt.cm.gray_r, interpolation='nearest')
axs[1].set_axis_off()
axs[1].imshow(digits.images[2], cmap=plt.cm.gray_r, interpolation='nearest')
plt.show()



Ridimensionamento su 4 dimensioni e normalizzazione e standardizzazione dei dati

In [ ]:
N_DIM = 4
TRAIN_SIZE = 100
TEST_SIZE = 20


sample_train, sample_test, label_train, label_test = train_test_split(
    digits.data, 
    digits.target, 
    test_size=0.2, 
    random_state=22
    )
    

pca = PCA(n_components=N_DIM).fit(sample_train)
sample_train = pca.transform(sample_train)
sample_test = pca.transform(sample_test)

std_scale = StandardScaler().fit(TRAIN)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)

samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)


sample_train = sample_train[:TRAIN_SIZE]
label_train = label_train[:TRAIN_SIZE]

sample_test = sample_test[:TEST_SIZE]
label_test = label_test[:TEST_SIZE]

Viene mostrata la mappa di encoding dei dati (non verrà usata questa nelle sezioni successive)

In [ ]:
encode_map = ZZFeatureMap(feature_dimension=N_DIM, reps=1, entanglement='linear', insert_barriers=True)
encode_circuit = encode_map.bind_parameters(sample_train[0])
encode_circuit.decompose().draw(output='mpl')

Definita la feature map ZZ per l'encoding dei dati, creiamo il quantum kernel sulla base di questa feature map. Il quantum kernel di occuperà di definire un circuito che effettua l'inner product della feature map per la sua coniugata trasposta.

In [ ]:
zz_map = ZZFeatureMap(feature_dimension=4, reps=2, entanglement='linear', insert_barriers=True)
zz_kernel = QuantumKernel(feature_map=zz_map, quantum_instance=Aer.get_backend('statevector_simulator'))

zz_circuit = zz_kernel.construct_circuit(sample_train[6], sample_train[7])
zz_circuit.decompose().decompose().draw(output='mpl')

Calcolata la feature map e il kernel, possiamo facilemente visualizzare la correlazione tra due istanze del dataset. Se due vettori sono fortemente correlati dato in input il basis state 0 mi aspetto di trovare in output il basis state 0 (un numero maggiore di volte). Abbiamo infatti due gate hadamard che riportano lo stato dalla equisuperposition ad una superposition. Se sono correlati la trasformazione delle feature map non modificherà troppo le amplitude e produrrà lo stesso output. Prendiamo infatti come similarità (espressa come probabilità). La frequenza di output dello stato 0. Indici di esempio


In [ ]:
ID_A = 0
ID_B = 10

zz_map = ZZFeatureMap(feature_dimension=4, reps=1, entanglement='linear', insert_barriers=True)
zz_kernel = QuantumKernel(feature_map=zz_map, quantum_instance=Aer.get_backend('statevector_simulator'))

zz_circuit = zz_kernel.construct_circuit(sample_train[ID_A], sample_train[ID_B])
zz_circuit.decompose().decompose().draw(output='mpl')

backend = Aer.get_backend('qasm_simulator')
job = execute(zz_circuit, backend, seed_transpiler=1024)
counts = job.result().get_counts(zz_circuit)

print(f"{'A:':<5s}{sample_train[ID_A]}")
print(f"{'B:':<5s}{sample_train[ID_B]}")
print(f"{'A~B:':<5s}{(counts['0'*N_DIM]/sum(counts.values())) * 100:04.2f}%")

plot_histogram(counts)


Viene calcolata quindi la matrice kernel sulla base delle similarità calcolate come nel codice precedente

In [ ]:
matrix_train = zz_kernel.evaluate(x_vec=sample_train) 
matrix_test = zz_kernel.evaluate(x_vec=sample_test, y_vec=sample_train)


fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.asmatrix(matrix_train), interpolation='nearest', origin='upper', cmap='Blues')
axs[0].set_title("Training Kernel Matrix")
axs[1].imshow(np.asmatrix(matrix_test), interpolation='nearest', origin='upper', cmap='Reds')
axs[1].set_title("Testing Kernel Matrix")
plt.show()

## Training e testing 

In [ ]:
quantum_svm = SVC(kernel=zz_kernel.evaluate)
quantum_svm.fit(sample_train, label_train)
quantum_svm_score = quantum_svm.score(sample_test, label_test)
print(f"Quantum SVM (ZZ Encoding) score: {quantum_svm_score}")

In [ ]:
quantum_svm = SVC(kernel='precomputed')
quantum_svm.fit(matrix_train, label_train)
quantum_svm_score = quantum_svm.score(matrix_test, label_test)
print(f"Quantum SVM (ZZ Encoding) score: {quantum_svm_score}")

# Neural Network
Creazione del dataset, feature randomiche con valori tra -1 e +1, classe 0 se la somma è minore di 0, classe 1 se la somma è maggiore di 0
La rete utilizzata utilizza:

- COBYLA come ottimizzatore
- RealAmplitude come encoder (amplitude encoding) e implementazione del circuito variazionale (ansatz, circuito parametrizzato, implementazione della rete neurale nel nostro caso)

In [ ]:
DATA_SIZE = 40
NUM_FEATURES  = 2
TRAIN_SIZE = 30

DATA = 2 * algorithm_globals.random.random([DATA_SIZE, NUM_FEATURES]) - 1
LABELS = 1 * (np.sum(DATA, axis=1) > 0)

DATA = MinMaxScaler().fit_transform(DATA)
LABELS = OneHotEncoder(sparse=False).fit_transform(LABELS.reshape(-1,1))

print(DATA.shape)
print(LABELS.shape)
print(DATA[0:5, :],'\n',LABELS[0:5, :])

train_data, test_data, train_labels, test_labels = train_test_split(DATA, LABELS, train_size=TRAIN_SIZE, random_state=algorithm_globals.random_seed)

In [ ]:
OBJECTIVE_VALUES = []
IT = 100
def callback_graph(_, objective_value):
    clear_output(wait=True)
    OBJECTIVE_VALUES.append(objective_value)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    stage1_len = np.min((len(OBJECTIVE_VALUES), IT))
    stage1_x = np.linspace(1, stage1_len, stage1_len)
    stage1_y = OBJECTIVE_VALUES[:stage1_len]
    plt.plot(stage1_x, stage1_y, color="orange")
    plt.show()

In [ ]:
OBJECTIVE_VALUES = []
optimizer = COBYLA(maxiter=IT)
ansatz = RealAmplitudes(NUM_FEATURES)
model = VQC(ansatz=ansatz, optimizer=optimizer, initial_point=[0.5] * ansatz.num_parameters, callback=callback_graph)
model.fit(train_data, train_labels)

print("Train score : ", model.score(train_data, train_labels))
print("Test score  : ", model.score(test_data, test_labels))


# CNN

## Conv e Pool Layer

Layer convoluzione con $N(\alpha, \beta, \gamma) \subset SU(n)$ (Special Unitary Groups of Degree n).

In [ ]:
def conv_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)
    target.cx(1, 0)
    target.rz(np.pi / 2, 0)
    return target


def pool_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)
    return target


def conv_layer(n_qbit, params_prefix, name="conv_layer"):

    qc = QuantumCircuit(n_qbit, name=name)
    qbits = list(range(n_qbit))
    params = ParameterVector(params_prefix, n_qbit * 3)
    
    if n_qbit > 2:
        for index, couple in enumerate(zip(qbits, qbits[1:] + [0])):
            qc = qc.compose(conv_circuit(params[index*3 : (index*3)+3]), [couple[0],couple[1]])
            qc.barrier()
    else:
        qc = qc.compose(conv_circuit(params[0 : 3]), [0,1])
        qc.barrier()

    qc_inst = qc.to_instruction()
    qc = QuantumCircuit(n_qbit)
    qc.append(qc_inst, qbits)

    return qc


def pool_layer(sources, sinks, params_prefix, name="pool_layer"):
    n_qbit = len(sources) + len(sinks)
    qc = QuantumCircuit(n_qbit, name=name)
    params = ParameterVector(params_prefix, n_qbit // 2 * 3)

    for index, couple in enumerate(zip(sources, sinks)):
        qc = qc.compose(pool_circuit(params[index*3 : (index*3) + 3]), [couple[0], couple[1]])
        qc.barrier()

    qc_inst = qc.to_instruction()
    qc = QuantumCircuit(n_qbit)
    qc.append(qc_inst, range(n_qbit))

    return qc

    




N_QBIT = 4

print("CONV CIRCUIT")
display(conv_circuit([1,1,1]).draw(output="mpl"))
print("POOL CIRCUIT")
display(pool_circuit([1,1,1]).draw(output="mpl"))
print("CONV LAYER")
display(conv_layer(N_QBIT, '\u03B8').draw(output="mpl"))
display(conv_layer(N_QBIT, '\u03B8').decompose().draw(output="mpl"))


print("POOL LAYER")
display(pool_layer([0,1], [2,3], '\u03B8').draw(output="mpl"))
display(pool_layer([0,1], [2,3], '\u03B8').decompose().draw(output="mpl"))

## Custom CNN

In [ ]:
digits = datasets.load_digits(n_class=2)

fig, axs = plt.subplots(1, 2, figsize=(6,3))
axs[0].set_axis_off()
axs[0].imshow(digits.images[0], cmap=plt.cm.gray_r, interpolation='nearest')
axs[1].set_axis_off()
axs[1].imshow(digits.images[2], cmap=plt.cm.gray_r, interpolation='nearest')
plt.show()


resize_data = [transform.resize(x, (6,6), mode='constant').ravel() for x in digits.images]

plt.imshow(resize_data[0].reshape((6,6)))


In [ ]:
N_CLASSES = 2
N_TRAIN = 20
STANDARDIZE = False
MINMAXSCALE = False

digits = datasets.load_digits(n_class=N_CLASSES)

resize_data = np.array([transform.resize(x, (4,4), mode='constant').ravel() for x in digits.images])


TRAIN, TEST, TRAIN_LABELS, TEST_LABELS = train_test_split(
    resize_data, 
    np.array([1 if x else -1 for x in digits.target]), 
    test_size=0.3, 
    random_state=22
    )

    
if STANDARDIZE:
    std_scale = StandardScaler().fit(TRAIN)
    TRAIN = std_scale.transform(TRAIN)
    TEST = std_scale.transform(TEST)

if MINMAXSCALE:
    samples = np.append(TRAIN, TEST, axis=0)
    minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
    TRAIN = minmax_scale.transform(TRAIN)
    TEST = minmax_scale.transform(TEST)


TRAIN = TRAIN[:N_TRAIN,:]
TRAIN_LABELS = TRAIN_LABELS[:N_TRAIN]


TRAIN.shape

In [ ]:
def generate_dataset(num_images):
    images = []
    labels = []
    hor_array = np.zeros((6, 8))
    ver_array = np.zeros((4, 8))

    j = 0
    for i in range(0, 7):
        if i != 3:
            hor_array[j][i] = np.pi / 2
            hor_array[j][i + 1] = np.pi / 2
            j += 1

    j = 0
    for i in range(0, 4):
        ver_array[j][i] = np.pi / 2
        ver_array[j][i + 4] = np.pi / 2
        j += 1

    for n in range(num_images):
        rng = algorithm_globals.random.integers(0, 2)
        if rng == 0:
            labels.append(-1)
            random_image = algorithm_globals.random.integers(0, 6)
            images.append(np.array(hor_array[random_image]))
        elif rng == 1:
            labels.append(1)
            random_image = algorithm_globals.random.integers(0, 4)
            images.append(np.array(ver_array[random_image]))

        # Create noise
        for i in range(8):
            if images[-1][i] == 0:
                images[-1][i] = algorithm_globals.random.uniform(0, np.pi / 4)
    return images, labels

images, labels = generate_dataset(50)

TRAIN, TEST, TRAIN_LABELS, TEST_LABELS = train_test_split(
    images, labels, test_size=0.3
)



In [ ]:
print(f"{'DATA SHAPE':15s}: {np.array(TRAIN).shape}")

N_QBIT = np.array(TRAIN).shape[1]

feature_map = ZFeatureMap(N_QBIT)
ansatz = QuantumCircuit(N_QBIT, name="Ansatz")

current_qbit = N_QBIT
index = 1

while(current_qbit >= 2):
    ansatz.compose(conv_layer(current_qbit, f'c{index}'), list(range(N_QBIT - current_qbit, N_QBIT)), inplace=True)
    ansatz.compose(pool_layer(list(range(0, current_qbit // 2)), list(range(current_qbit // 2 , current_qbit)), f'p{index}'), list(range(N_QBIT - current_qbit, N_QBIT)), inplace=True)
    current_qbit = current_qbit // 2
    index += 1


circuit = QuantumCircuit(N_QBIT)
circuit.compose(feature_map, range(N_QBIT), inplace=True)
circuit.compose(ansatz, range(N_QBIT), inplace=True)

print("\nQCNN Created:")
print(f"{'Input':15s}: {N_QBIT}")
print(f"{'Depth':15s}: {index}")

In [ ]:
ansatz.draw(output="mpl", filename="circuit.png")
ansatz.decompose().draw(output="mpl", filename="circuit_decomposed.png")
print()

In [ ]:
def callback_print(_, objective_value):
    global losses
    losses.append(objective_value)
    #print(f"Epoch: [{len(losses):3d}]  Losss: {objective_value:4.3f}")
    
observable = SparsePauliOp.from_list([("Z" + "I" * (N_QBIT - 1), 1)])

qnn = EstimatorQNN(
    circuit=circuit.decompose(),
    observables=observable,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
)

initial_point = np.asarray([0.5] * ansatz.num_parameters)

classifier = NeuralNetworkClassifier(
    qnn,
    optimizer=COBYLA(maxiter=400),
    callback=callback_print,
    initial_point=initial_point,
)

In [ ]:
x = np.asarray(TRAIN)
y = np.asarray(TRAIN_LABELS)

global losses
losses = []
classifier.fit(x, y)

In [ ]:
plt.plot(list(range(1,len(losses)+1)), losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("QCNN Training")
print(f"Accuracy from the train data : {np.round(100 * classifier.score(x, y), 2)}%")
y_predict = classifier.predict(TEST)
tx = np.asarray(TEST)
ty = np.asarray(TEST_LABELS)
print(f"Accuracy from the test data : {np.round(100 * classifier.score(tx, ty), 2)}%")